![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split
from datetime import datetime
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [80]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [81]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [82]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [83]:
df_train.info()

In [84]:
df_train.head(20)

In [85]:
df_test.info()

In [86]:
df_test.head(5)

In [87]:
sample_submission.head(5)

In [88]:
sample_submission.info()

In [89]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [90]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [91]:
data.sample(5)

In [92]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 


## Number of Reviews

In [93]:
# Для примера я возьму столбец Number of Reviews
data['Number of Reviews NA'] = data['Number of Reviews'].apply(lambda x: 1 if pd.isna(x) else 0)
# заменим NaN для пропусков в 'Number of Reviews' на 0 
data['Number of Reviews'].fillna(0, inplace=True)

In [94]:
display(data.loc[:, ['Number of Reviews', 'Number of Reviews NA']].sample(5))
display(data.loc[:, ['Number of Reviews', 'Number of Reviews NA']].info())

## Price Range

In [95]:
data['Price Range NA'] = data['Price Range'].apply(lambda x: 1 if pd.isna(x) else 0)
# для заполнения выберем наболее распространенную категорию $$ - $$$
display(data['Price Range'].value_counts())
print('price_range mode:', data['Price Range'].mode()[0])

In [96]:
data['Price Range'].fillna(data['Price Range'].mode()[0], inplace=True)
display(data.loc[:, ['Price Range', 'Price Range NA']].sample(5))
display(data.loc[:, ['Price Range', 'Price Range NA']].info())

## Cuisine Style

In [97]:
data['Cuisine Style NA'] = data['Cuisine Style'].apply(lambda x: 1 if pd.isna(x) else 0)
data['Cuisine Style NA'].value_counts()
# для заполнения пустых данных  в cuisine недостатчно информации 11590 пропусков

In [98]:
display(data.loc[:, ['Cuisine Style', 'Cuisine Style NA']].sample(5))
display(data.loc[:, ['Cuisine Style', 'Cuisine Style NA']].info())


## reviews

In [99]:
data['Reviews NA'] = data['Reviews'].apply(lambda x: 1 if pd.isna(x) else 0)
data['Reviews NA'].value_counts()
# для заполнения пустых данных  в reviews недостатчно информации 

In [100]:
data['Reviews'][1]

In [101]:
sum(data['Reviews'].isna())
#пропуски

In [102]:
data['Reviews'].fillna('[[], []]', inplace=True)


Идей особо не было как использовать данный признак. Пожтому я подсмотрел идею, как использовать даты данного признака.

Текст ревью в данном случае использовать не будем, а на основе дат добавим признаки количества дней между двумя послденими ревью и количество дней с момента последнего ревью


In [103]:
# паттерн для поиска дат
pattern = re.compile('\'\d+\/\d+\/\d+\'')
dates = data['Reviews'].apply(pattern.findall)
# врeменные признаки
data['date1'] = pd.to_datetime(dates.apply(lambda x: x[0] if len(x) > 0 else None))
data['date2'] = pd.to_datetime(dates.apply(lambda x: x[1] if len(x) > 1 else None))

display(data.loc[:, ['date1', 'date2']].sample(5))

### reviews_day_delta & reviews_days_since

In [104]:
def days_since_last_review(r):
    '''
    Считает количество дней с последней даты ревью данной записи
    датасета. Учитывает отсутствие одной из дат и пропуск (возвращает
    0 в последнем случае)
    '''
    datetime_now = datetime.now();
    if pd.notna(r.date1) and pd.notna(r.date2):
        if r.date1 > r.date2:
            return (datetime_now - r.date1).days
        else:
            return (datetime_now - r.date2).days
    else:
        if pd.notna(r.date1):
            return (datetime_now - r.date1).days
        if pd.notna(r.date2):
            return (datetime_now - r.date2).days        
    return 0 


def reviews_days_delta(r):
    '''
    Возвращает количество дней между двумя датами ревью в данной записи
    датасета. Учитывает отсутствие (одной из) дат - возвращает 0 в данном
    случае.
    '''
    if pd.notna(r.date1) and pd.notna(r.date2):
        return abs((r.date1 - r.date2).days)
    else:
        return 0

# новые признаки на основе дат последних ревью: количество дней между
# двумя последними ревью и количество дней со времени последнего ревью
data['Reviews Days Delta'] = data.apply(lambda row: reviews_days_delta(row), axis=1)
data['Reviews Days Since'] = data.apply(lambda row: days_since_last_review(row), axis=1)
# удаляем временные признаки
data.drop(['date1', 'date2'], inplace=True, axis=1)

In [105]:
display(data.sample(5))

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [106]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)


## Restaurant_id

Во вводном разборе было акцентировано внимание на том, что с данным признаком надо разобраться. Посмотрим, уникальны ли айди.


In [107]:
data['Restaurant_id'].value_counts()



Мы видим, что имеются повторы. Возможно, это сеть.

## ID_TA

In [108]:
data['ID_TA'].value_counts()
# Отметим не уникальность данного признака

## URL_TA

In [109]:
data['URL_TA'].value_counts()
# Так же увидели, что признак не уникальный

## City

In [110]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. 
#Особенно радует параметр dummy_na
# Сохраним названия городов - возможно, используем позже (get_dummies это делать, похоже, не умеет)
data['City_name'] = data['City']
data = pd.get_dummies(data, columns=['City'], dummy_na=False)


In [111]:
data.head(5)

In [112]:
data.sample(5)

## Price Range.

In [113]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [114]:
# Переводим столбец с ценовой категорией в числовой формат

data['Price Range'] = data['Price Range'].apply(lambda x: 3 if x == '$$$$' else x)
data['Price Range'] = data['Price Range'].apply(lambda x: 2 if x == '$$ - $$$' else x)
data['Price Range'] = data['Price Range'].apply(lambda x: 1 if x == '$' else x)

In [115]:
data['Price Range'].value_counts()

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

## Ranking

### Ranking To Rest Num & Ranking To Population

In [116]:
#население территории города (metro area) - источник: wiki
city_population = {'Paris':13024, 'Stockholm':2391, 'London':14257, 
                   'Berlin':6144, 'Munich':5991, 'Oporto':1721,
                   'Milan':4336, 'Bratislava':659, 'Vienna':2600, 
                   'Rome':4342, 'Barcelona':5474, 'Madrid':6791,
                   'Dublin':1417, 'Brussels':2500, 'Zurich':415, 
                   'Warsaw':3100, 'Budapest':3011, 'Copenhagen':2057,
                   'Amsterdam':2480, 'Lyon':2323, 'Hamburg':5107, 
                   'Lisbon':2827, 'Prague':2677, 'Oslo':1588,
                   'Helsinki':1525, 'Edinburgh':901, 'Geneva':201, 
                   'Ljubljana':537, 'Athens':2928,'Luxembourg':633, 
                   'Krakow':1752}

#количество ресторанов в городе - источник: датасет
rest_per_city = data.City_name.value_counts().to_dict()

data['Ranking To Rest Num'] = data.apply(lambda r: (r['Ranking'] / rest_per_city[r.City_name]), axis=1)
data['Ranking To Population'] = data.apply(lambda r: (r['Ranking'] / city_population[r.City_name]), axis=1)
data.sample(5)

### Number Of Reviews

Так же добавим признак выражающий отношение количества ревью к населению 


In [117]:
data['Reviews Num To Population'] = data.apply(lambda r: (r['Number of Reviews'] / city_population[r.City_name]), axis=1)


### Population

Почему бы не добавить и население города как признак


In [118]:
data['Population'] = data.City_name.apply(lambda x: (city_population[x]))

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [119]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [120]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [121]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [122]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [123]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [124]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [125]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [126]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [127]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями
data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [128]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews NA'] = df_output['Number of Reviews'].apply(lambda x: 1 if pd.isna(x) else 0)
    # заменим NaN для пропусков в 'Number of Reviews' на 0 как наиболее логичное значение 
    # для пропуска в данном случае
    df_output['Number of Reviews'].fillna(0, inplace=True)  
    
    df_output['Price Range NA'] = df_output['Price Range'].apply(lambda x: 1 if pd.isna(x) else 0)
    
      # признак содержит ординальные данные определяющие ценовую категорию ресторана, для 
    # заполнения выберем наболее распространенную категорию - среднюю
    df_output['Price Range'].fillna(df_output['Price Range'].mode()[0], inplace=True)
    df_output['Cuisine Style NA'] = df_output['Cuisine Style'].apply(lambda x: 1 if pd.isna(x) else 0)
    # не меняем пропуски в cuisine 
    df_output['Cuisine Style'].fillna('[Unspecified]')
    
    df_output['Reviews NA'] = df_output['Reviews'].apply(lambda x: 1 if pd.isna(x) else 0)
    df_output['Reviews'].fillna('[[], []]', inplace=True)
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    # Сохраним названия городов - возможно, используем позже (get_dummies это делать, похоже, не умеет)
    df_output['City_name'] = df_output['City']
    df_output = pd.get_dummies(df_output, columns=['City'], dummy_na=False)
    
    def price_range_to_num(val):
        '''
        Конвертирует строковое значение ценового диапазона на
        числовое, e.g.:
        $:                   1 
        $$-$$$:              2
        в остальных случаях: 3
        Returns:
            Числовое значение соответствующее ценовому диапазону
        '''
        if val == '$':        
            return 1
        elif val == '$$ - $$$':
            return 2
        elif val == '$$$$':
            return 3
        else:
            val
        
    df_output['Price Range'] = df_output['Price Range'].apply(price_range_to_num)
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    pattern = re.compile('\'\d+\/\d+\/\d+\'')
    dates = df_output['Reviews'].apply(pattern.findall)
    # врeменные признаки
    df_output['date1'] = pd.to_datetime(dates.apply(lambda x: x[0] if len(x) > 0 else None))
    df_output['date2'] = pd.to_datetime(dates.apply(lambda x: x[1] if len(x) > 1 else None))
    
    def days_since_last_review(r):
        '''
        Считает количество дней с последней даты ревью данной записи
        датасета. Учитывает отсутствие одной из дат и пропуск (возвращает
        0 в последнем случае)
        '''
        datetime_now = datetime.now();
        if pd.notna(r.date1) and pd.notna(r.date2):
            if r.date1 > r.date2:
                return (datetime_now - r.date1).days
            else:
                return (datetime_now - r.date2).days
        else:
            if pd.notna(r.date1):
                return (datetime_now - r.date1).days
            if pd.notna(r.date2):
                return (datetime_now - r.date2).days        
        return 0 


    def reviews_days_delta(r):
        '''
        Возвращает количество дней между двумя датами ревью в данной записи
        датасета. Учитывает отсутствие (одной из) дат - возвращает 0 в данном
        случае.
        '''
        if pd.notna(r.date1) and pd.notna(r.date2):
            return abs((r.date1 - r.date2).days)
        else:
            return 0
    
    df_output['Reviews Days Delta'] = df_output.apply(lambda row: reviews_days_delta(row), axis=1)
    df_output['Reviews Days Since'] = df_output.apply(lambda row: days_since_last_review(row), axis=1)
    # удаляем временные признаки
    df_output.drop(['date1', 'date2'], inplace=True, axis=1)
    
    #население территории города (metro area) - источник: wiki
    city_population = {'Paris':13024, 'Stockholm':2391, 'London':14257, 
                       'Berlin':6144, 'Munich':5991, 'Oporto':1721,
                       'Milan':4336, 'Bratislava':659, 'Vienna':2600, 
                       'Rome':4342, 'Barcelona':5474, 'Madrid':6791,
                       'Dublin':1417, 'Brussels':2500, 'Zurich':415, 
                       'Warsaw':3100, 'Budapest':3011, 'Copenhagen':2057,
                       'Amsterdam':2480, 'Lyon':2323, 'Hamburg':5107, 
                       'Lisbon':2827, 'Prague':2677, 'Oslo':1588,
                       'Helsinki':1525, 'Edinburgh':901, 'Geneva':201, 
                       'Ljubljana':537, 'Athens':2928,'Luxembourg':633, 
                       'Krakow':1752}
    
    # количество ресторанов в городе - источник: датасет
    rest_per_city = df_output.City_name.value_counts().to_dict()
    
    df_output['Ranking To Rest Num'] = df_output.apply(lambda r: (r['Ranking'] / rest_per_city[r.City_name]), axis=1)
    df_output['Ranking To Population'] = df_output.apply(lambda r: (r['Ranking'] / city_population[r.City_name]), axis=1) 
    df_output['Reviews Num To Population'] = df_output.apply(lambda r: (r['Number of Reviews'] / city_population[r.City_name]), axis=1)
    df_output['Population'] = df_output.City_name.apply(lambda x: (city_population[x]))
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [129]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [130]:
df_preproc.info()

In [131]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [132]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [133]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [134]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [135]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [136]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [137]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [138]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [139]:
test_data.sample(10)

In [140]:
test_data = test_data.drop(['Rating'], axis=1)

In [141]:
sample_submission

In [142]:
predict_submission = model.predict(test_data)

In [143]:
predict_submission

In [144]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
